In [1]:
import arviz as az
import bambi as bmb
import numpy as np
import pandas as pd
import pytensor

import hssm
import ssms.basic_simulators

pytensor.config.floatX = "float32"

### Using include paramater to use regression and update priors.

#### Case 1: Regression type formula. 

In [2]:
# get some fake simulation data
intercept = 0.3
x = np.random.uniform(0.2, 0.5, size=1000)
y = np.random.uniform(0.1, 0.4, size=1000)

v = intercept + 0.8 * x + 0.3 * y

In [3]:
true_values = np.column_stack(
    [v, np.repeat([[1.5, 0.5, 0.5, 0.0]], axis=0, repeats=1000)]
)
true_values.shape

(1000, 5)

In [4]:
obs_ddm_reg_v = ssms.basic_simulators.simulator(true_values, model="ddm", n_samples=1)
obs_ddm_reg_v

dataset_reg_v = pd.DataFrame(
    {
        "rt": obs_ddm_reg_v["rts"].flatten(),
        "response": obs_ddm_reg_v["choices"].flatten(),
        "x": x,
        "y": y,
    }
)

dataset_reg_v

,rt,response,x,y
0,3.202986,1,0.413635,0.202105
1,1.773003,1,0.361816,0.154223
2,2.144021,1,0.216562,0.316722
3,8.406610,1,0.442631,0.194695
4,1.895009,1,0.297407,0.142648
...,...,...,...,...
995,1.422992,1,0.332995,0.332539
996,3.303979,1,0.310310,0.212713
997,1.640997,1,0.202920,0.353149
998,1.836006,-1,0.421434,0.244051


- v is parent
- x is a feature from the dataset_reg_v
- y is a feature from the dataset_reg_v

In [5]:
loglik = hssm.wfpt.WFPT

priors = {
    "c(rt, response)": {
        "Intercept": bmb.Prior("Uniform", lower=0.0, upper=0.5),
        "x": bmb.Prior("Uniform", lower=0.0, upper=1.0),
        "y": bmb.Prior("Uniform", lower=0.0, upper=1.0),
    },
    "a": bmb.Prior("Uniform", lower=0.3, upper=2.5),
    "z": bmb.Prior("Uniform", lower=0.1, upper=0.9),
    "t": bmb.Prior("Uniform", lower=0.0, upper=2.0),
}

links = {"v": "identity"}

In [6]:
likelihood = bmb.Likelihood(
    "WFPT_Loglik", params=["v", "a", "z", "t"], parent="v", dist=loglik
)

In [7]:
family = bmb.Family("WFPT_Family", likelihood=likelihood, link=links)

In [8]:
bambi_model = bmb.Model(
    "c(rt, response) ~ 1 + x + y", data=dataset_reg_v, family=family, priors=priors
)

In [14]:
sample = bambi_model.fit(
    cores=1,
    chains=2,
    draws=500,
)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (2 chains in 1 job)
NUTS: [c(rt, response)_z, c(rt, response)_t, c(rt, response)_a, Intercept, x, y]


Sampling 2 chains for 1_000 tune and 500 draw iterations (2_000 + 1_000 draws total) took 20 seconds.
We recommend running at least 4 chains for robust computation of convergence diagnostics


In [17]:
# Make some new data points
new_data = pd.DataFrame(
    {
        "x": np.random.uniform(0.2, 0.5, size=10),
        "y": np.random.uniform(0.1, 0.4, size=10),
    }
)

In [18]:
# posterior predictive sampling

bambi_model.predict(sample, data=new_data, inplace=False, kind="pps")

ValueError: coordinate chain has dimensions ('chain',), but these are not a subset of the DataArray dimensions ('dim_0', 'dim_1', 'dim_2', 'dim_3')

In [21]:
# set the model family to be multivariate

setattr(bambi_model.family, "KIND", "Multivariate")

In [22]:
# Different Error

bambi_model.predict(sample, data=new_data, inplace=False, kind="pps")

KeyError: 'c(rt, response)_dim'

In [23]:
multivariate_family = bmb.families.multivariate.MultivariateFamily(
    "WFPT_Family", likelihood=likelihood, link=links
)

In [24]:
bambi_model_multi = bmb.Model(
    "c(rt, response) ~ 1 + x + y",
    data=dataset_reg_v,
    family=multivariate_family,
    priors=priors,
)

In [26]:
bambi_model_multi.fit(cores=1, chains=2, draws=500)

IndexError: too many indices for array

In [5]:
model_reg_v = hssm.HSSM(
    data=dataset_reg_v,
    include=[
        {
            "name": "v",
            "prior": {
                "Intercept": {"name": "Uniform", "lower": 0.0, "upper": 0.5},
                "x": {"name": "Uniform", "lower": 0.0, "upper": 1.0},
                "y": {"name": "Uniform", "lower": 0.0, "upper": 1.0},
            },
            "formula": "v ~ 1 + x + y",
            "link": "identity",
        }
    ],
)
model_reg_v

Hierarchical Sequential Sampling Model
Model: ddm

Response variable: rt,response
Observations: 1000

Parameters:

v ~ 1 + x + y
	Link: identity
	bounds: (-3.0, 3.0)
	Intercept ~ Uniform(lower: 0.0, upper: 0.5)
	x ~ Uniform(lower: 0.0, upper: 1.0)
	y ~ Uniform(lower: 0.0, upper: 1.0)
a ~ Uniform(lower: 0.30000001192092896, upper: 2.5)	bounds: (0.3, 2.5)
z ~ Uniform(lower: 0.10000000149011612, upper: 0.8999999761581421)	bounds: (0.1, 0.9)
t ~ Uniform(lower: 0.0, upper: 2.0)	bounds: (0.0, 2.0)

In [ ]:
trace_reg_v = model_reg_v.sample(cores=2, chains=2, draws=500)

In [ ]:
trace_reg_v

In [ ]:
az.plot_trace(model_reg_v.traces)

In [ ]:
# Looks like parameter recovery was successful
az.summary(model_reg_v.traces)

In [ ]:
setattr(model_reg_v.model.family, "KIND", "Multivariate")

In [ ]:
delattr(model_reg_v.model.family, "KIND")

In [ ]:
posterior = model_reg_v.model.predict(
    model_reg_v.traces, data=new_data.iloc[1:10, :], inplace=False, kind="pps"
)

In [ ]:
from copy import deepcopy
from bambi.utils import get_aliased_name
import pymc as pm
import xarray as xr

In [ ]:
def posterior_predictive(model, posterior, **kwargs):
    """Get draws from the posterior predictive distribution

    This function works for almost all the families. It grabs the draws for the parameters
    needed in the response distribution, and then gets samples from the posterior predictive
    distribution using `pm.draw()`. It won't work when the response distribution requires
    parameters that are not available in `posterior`.

    Parameters
    ----------
    model : bambi.Model
        The model
    posterior : xr.Dataset
        The xarray dataset that contains the draws for all the parameters in the posterior.
        It must contain the parameters that are needed in the distribution of the response, or
        the parameters that allow to derive them.
    kwargs :
        Parameters that are used to get draws but do not appear in the posterior object or
        other configuration parameters.
        For instance, the 'n' in binomial models and multinomial models.

    Returns
    -------
    xr.DataArray
        A data array with the draws from the posterior predictive distribution
    """
    response_dist = get_response_dist(model.family)
    params = model.family.likelihood.params
    response_aliased_name = get_aliased_name(model.response_component.response_term)

    kwargs.pop("data", None)  # Remove the 'data' kwarg
    dont_reshape = kwargs.pop("dont_reshape", [])
    output_dataset_list = []

    # In the posterior xr.Dataset we need to consider aliases,
    # but we don't use aliases when passing kwargs to the PyMC distribution.
    for param in params:
        # Extract posterior draws for the parent parameter
        if param == model.family.likelihood.parent:
            component = model.components[model.response_name]
            var_name = response_aliased_name + "_mean"
            kwargs[param] = posterior[var_name].to_numpy()
            output_dataset_list.append(posterior[var_name])
        else:
            # Extract posterior draws for non-parent parameters
            component = model.components[param]
            if component.alias:
                var_name = component.alias
            else:
                var_name = f"{response_aliased_name}_{param}"

            if var_name in posterior:
                kwargs[param] = posterior[var_name].to_numpy()
                output_dataset_list.append(posterior[var_name])
            elif hasattr(component, "prior") and isinstance(
                component.prior, (int, float)
            ):
                kwargs[param] = np.asarray(component.prior)
            else:
                raise ValueError(
                    "Non-parent parameter not found in posterior."
                    "This error shouldn't have happened!"
                )

    # Determine the array with largest number of dimensions
    ndims_max = max(x.ndim for x in kwargs.values())

    # Append a dimension when needed. Required to make `pm.draw()` work.
    # However, some distributions like Multinomial, require some parameters to be of a smaller
    # dimension than others (n.ndim <= p.ndim - 1) so we don't reshape those.
    for key, values in kwargs.items():
        if key in dont_reshape:
            continue
        kwargs[key] = expand_array(values, ndims_max)

    if hasattr(model.family, "transform_kwargs"):
        kwargs = model.family.transform_kwargs(kwargs)

    output_array = pm.draw(response_dist.dist(**kwargs))
    output_coords_all = xr.merge(output_dataset_list).coords

    coord_names = ["chain", "draw", response_aliased_name + "_obs"]
    is_multivariate = (
        hasattr(model.family, "KIND") and model.family.KIND == "Multivariate"
    )
    if is_multivariate:
        coord_names.append(response_aliased_name + "_dim")

    output_coords = {}
    for coord_name in coord_names:
        output_coords[coord_name] = output_coords_all[coord_name]
    return xr.DataArray(output_array, coords=output_coords)


def get_response_dist(family):
    """Get the PyMC distribution for the response

    Parameters
    ----------
    family : bambi.Family
        The family for which the response distribution is wanted

    Returns
    -------
    pm.Distribution
        The response distribution
    """
    mapping = {"Cumulative": pm.Categorical, "StoppingRatio": pm.Categorical}

    if family.likelihood.dist:
        dist = family.likelihood.dist
    elif family.likelihood.name in mapping:
        dist = mapping[family.likelihood.name]
    else:
        dist = getattr(pm, family.likelihood.name)
    return dist


def expand_array(x, ndim):
    """Add dimensions to an array to match the number of desired dimensions

    If x.ndim < ndim, it adds ndim - x.ndim dimensions after the last axis. If not, it is left
    untouched.

    For example, if we have a normal regression model with n = 1000, chains = 2, and draws = 500
    the shape of the draws of mu will be (2, 500, 1000) but the shape of the draws of sigma will be
    (2, 500). This function makes sure the shape of the draws of sigma is (2, 500, 1) which is
    comaptible with (2, 500, 1000).

    Parameters
    ----------
    x : np.ndarray
        The array
    ndim : int
        The number of desired dimensions

    Returns
    -------
    np.ndarray
        The array with the expanded dimensions
    """
    if x.ndim == ndim:
        return x
    dims_to_expand = tuple(range(ndim - 1, x.ndim - 1, -1))
    return np.expand_dims(x, dims_to_expand)

In [ ]:
def predict(
    model, idata, kind="mean", data=None, inplace=True, include_group_specific=True
):
    """Predict method for Bambi models

    Obtains in-sample and out-of-sample predictions from a fitted Bambi model.

    Parameters
    ----------
    idata : InferenceData
        The ``InferenceData`` instance returned by ``.fit()``.
    kind : str
        Indicates the type of prediction required. Can be ``"mean"`` or ``"pps"``. The
        first returns draws from the posterior distribution of the mean, while the latter
        returns the draws from the posterior predictive distribution
        (i.e. the posterior probability distribution for a new observation).
        Defaults to ``"mean"``.
    data : pandas.DataFrame or None
        An optional data frame with values for the predictors that are used to obtain
        out-of-sample predictions. If omitted, the original dataset is used.
    include_group_specific : bool
        If ``True`` make predictions including the group specific effects. Otherwise,
        predictions are made with common effects only (i.e. group specific are set
        to zero).
    inplace : bool
        If ``True`` it will modify ``idata`` in-place. Otherwise, it will return a copy of
        ``idata`` with the predictions added. If ``kind="mean"``, a new variable ending in
        ``"_mean"`` is added to the ``posterior`` group. If ``kind="pps"``, it appends a
        ``posterior_predictive`` group to ``idata``. If any of these already exist, it will be
        overwritten.

    Returns
    -------
    InferenceData or None
    """
    if kind not in ("mean", "pps"):
        raise ValueError("'kind' must be one of 'mean' or 'pps'")

    if not inplace:
        idata = deepcopy(idata)

    response_aliased_name = get_aliased_name(model.response_component.response_term)

    # ALWAYS predict the mean response
    means_dict = {}
    # To store the HSGP contributions that are also added to the posterior dataset
    hsgp_dict = {}
    response_dim = response_aliased_name + "_obs"
    for name, component in model.distributional_components.items():
        if name == model.response_name:
            var_name = response_aliased_name + "_mean"
        else:
            if component.alias:
                var_name = component.alias
            else:
                var_name = f"{response_aliased_name}_{name}"

        means_dict[var_name] = component.predict(
            idata, data, include_group_specific, hsgp_dict
        )

        # Drop var/dim if already present. Needed for out-of-sample predictions.
        if var_name in idata.posterior.data_vars:
            idata.posterior = idata.posterior.drop_vars(var_name)

    if response_dim in idata.posterior.dims:
        idata.posterior = idata.posterior.drop_dims(response_dim)

    # Use the first DataArray to get the number of observations
    obs_n = len(list(means_dict.values())[0].coords.get(response_dim))
    idata.posterior = idata.posterior.assign_coords({response_dim: list(range(obs_n))})

    for name, value in means_dict.items():
        idata.posterior[name] = value

    # Add HSGP contributions to the posterior dataset
    for component in model.distributional_components.values():
        for name, hsgp_contribution in hsgp_dict.items():
            term = component.hsgp_terms.get(name, None)
            if term is None:
                continue
            term_aliased_name = get_aliased_name(term)
            idata.posterior[term_aliased_name] = hsgp_contribution.transpose(
                "chain", "draw", ...
            )

    # Only if requested predict the predictive distribution
    if kind == "pps":
        required_kwargs = {"model": model, "posterior": idata.posterior}
        optional_kwargs = {"data": data}

        pps = posterior_predictive(**required_kwargs, **optional_kwargs)
        pps = pps.to_dataset(name=response_aliased_name)

        if "posterior_predictive" in idata:
            del idata.posterior_predictive
        idata.add_groups({"posterior_predictive": pps})
        idata.posterior_predictive = idata.posterior_predictive.assign_attrs(
            modeling_interface="bambi", modeling_interface_version=__version__
        )

    if inplace:
        return None
    else:
        return idata

In [ ]:
predict(
    model_reg_v.model,
    model_reg_v.traces,
    data=new_data.iloc[1:10, :],
    inplace=False,
    kind="pps",
)

In [ ]:
test_function(1)

In [ ]:
# simulator always expects a list or matrix

# preprocessing in simulator wrapper
# take care of special cases --> concatenate into matrix

# postprocessing in ...
# (chains, trial_output_dim, n_mcmc_samples, n_data)

In [ ]:
# Logic of output

# Definition single sample from posterior predictive (n_samples)

# where do the parameters come from --> Uniform sample from the trace!

# (n_chains, )
# a single posterior predictive sample has the same dimension as our data
# and is a simulated version of our dataset when fixing parameters of our model
# according to a single uniform draw from our posterior (that is already collected from the .sample() call)

#

# n samples form posterior predictive is NOT NECESSARILY == n_mcmc samples (which live in parameter space)